Analysis of JSON schema restricted output. For example the below prompt for A-D multiple choice question:

```
json_schema_prompt = """
Please answer the following question adhering to these format instructions:
The output should be formatted as a JSON instance that conforms to the JSON schema below.
 
{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "Answer": {
      "type": "string",
      "enum" : ["A", "B", "C", "D"]
    }
  },
  "required": [
    "Answer"
  ]
}

The output {"Answer": "A"} is a well-formatted instance of the schema, the output {"Answer": "E"} is not well-formatted. A string answer like "The correct answer is A" is not well-formatted.

The question is: 


"""
```

Code to run evaluation is in `python run_schema.py`, prompts for A-K and Yes-No are in that code. 

The `raw` runs being used for comparison come from `experiments/v2/runs` with the first 5 runs being copied over to the local `runs` directory. Note that to rerun you will have to move the `raw` directories and `schema` runs because the `run_experiments.py` script will not run if there is a matching directory for the task, model, number of shots and temperature. 


In [54]:
# Run evaluate.py on the directory 'runs'
os.system('python ../evaluate.py -d runs -npp')

stability_df = pd.read_csv('stability_eval.csv')

display(stability_df)

Trying runs/gpt-35-turbo-0.0-ruin_names-few_2025-02-17_10-58-10/gpt-35-turbo-0.0-ruin_names-few-1.csv
Trying runs/gpt-35-turbo-0.0-ruin_names-few_2025-02-17_10-58-10/gpt-35-turbo-0.0-ruin_names-few-0.csv
Trying runs/gpt-35-turbo-0.0-ruin_names-few_2025-02-17_10-58-10/gpt-35-turbo-0.0-ruin_names-few-2.csv
Trying runs/gpt-35-turbo-0.0-ruin_names-few_2025-02-17_10-58-10/gpt-35-turbo-0.0-ruin_names-few-3.csv
Trying runs/gpt-35-turbo-0.0-ruin_names-few_2025-02-17_10-58-10/gpt-35-turbo-0.0-ruin_names-few-4.csv
Trying runs/gpt-35-turbo-0.0-navigate-few_2025-02-17_12-20-38/gpt-35-turbo-0.0-navigate-few-4.csv
Trying runs/gpt-35-turbo-0.0-navigate-few_2025-02-17_12-20-38/gpt-35-turbo-0.0-navigate-few-2.csv
Trying runs/gpt-35-turbo-0.0-navigate-few_2025-02-17_12-20-38/gpt-35-turbo-0.0-navigate-few-3.csv
Trying runs/gpt-35-turbo-0.0-navigate-few_2025-02-17_12-20-38/gpt-35-turbo-0.0-navigate-few-1.csv
Trying runs/gpt-35-turbo-0.0-navigate-few_2025-02-17_12-20-38/gpt-35-turbo-0.0-navigate-few-0.csv


,Unnamed: 0,model,model_config,task,task_config,TACr,TARr,TACa,TARa,correct_count_per_run,...,num_questions,N,best_possible_count,best_possible_accuracy,worst_possible_count,worst_possible_accuracy,spread,bootstrap_counts,bootstrap_pcts,date
0,0,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",ruin_names,"{'prompt_type': 'v2', 'shots': 'few'}",225,0.900000,225,0.900000,"[110, 111, 109, 115, 112]",...,250,5,121,0.484000,105,0.420000,0.064000,"[107, 108, 109, 109, 111, 111, 112, 112, 113, ...","[0.428, 0.432, 0.436, 0.436, 0.444, 0.444, 0.4...",2025-02-17_10-58-10
1,1,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",logical_deduction,"{'prompt_type': 'v2', 'shots': 'few'}",236,0.944000,236,0.944000,"[107, 110, 109, 107, 107]",...,250,5,113,0.452000,102,0.408000,0.044000,"[107, 107, 107, 107, 108, 108, 108, 109, 109, ...","[0.428, 0.428, 0.428, 0.428, 0.432, 0.432, 0.4...",2025-02-17_13-17-09
2,2,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",professional_accounting,"{'prompt_type': 'v2', 'shots': 'few'}",245,0.868794,245,0.868794,"[92, 85, 82, 91, 87]",...,282,5,104,0.368794,77,0.273050,0.095745,"[86, 86, 87, 87, 88, 88, 88, 89, 90, 93]","[0.3049645390070922, 0.3049645390070922, 0.308...",2025-02-15_16-14-28
3,3,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",public_relations,"{'prompt_type': 'v2', 'shots': 'few'}",97,0.881818,97,0.881818,"[57, 56, 57, 59, 57]",...,110,5,63,0.572727,53,0.481818,0.090909,"[56, 57, 57, 58, 58, 58, 58, 58, 59, 61]","[0.509090909090909, 0.5181818181818182, 0.5181...",2025-02-17_11-34-50
4,4,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",high_school_european_history,"{'prompt_type': 'v2', 'shots': 'few'}",161,0.975758,161,0.975758,"[137, 134, 136, 135, 137]",...,165,5,137,0.830303,134,0.812121,0.018182,"[135, 135, 135, 136, 136, 136, 136, 136, 136, ...","[0.8181818181818182, 0.8181818181818182, 0.818...",2025-02-17_11-12-23
5,5,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",college_mathematics,"{'prompt_type': 'v2', 'shots': 'few'}",89,0.890000,89,0.890000,"[31, 32, 29, 29, 27]",...,100,5,33,0.330000,26,0.260000,0.070000,"[27, 28, 29, 29, 30, 30, 31, 31, 31, 31]","[0.27, 0.28, 0.29, 0.29, 0.3, 0.3, 0.31, 0.31,...",2025-02-15_16-27-59
6,6,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",navigate,"{'prompt_type': 'v2', 'shots': 'few'}",246,0.984000,246,0.984000,"[163, 161, 161, 160, 160]",...,250,5,164,0.656000,160,0.640000,0.016000,"[160, 160, 160, 160, 161, 161, 161, 161, 161, ...","[0.64, 0.64, 0.64, 0.64, 0.644, 0.644, 0.644, ...",2025-02-17_12-20-38
7,7,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0....",geometric_shapes,"{'prompt_type': 'v2', 'shots': 'few'}",221,0.884000,221,0.884000,"[83, 88, 85, 85, 86]",...,250,5,94,0.376000,75,0.300000,0.076000,"[82, 84, 85, 86, 86, 86, 87, 87, 88, 88]","[0.328, 0.336, 0.34, 0.344, 0.344, 0.344, 0.34...",2025-02-17_12-32-03
8,8,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0.0}",ruin_names,"{'prompt_type': 'v2', 'shots': 'few'}",176,0.704000,204,0.816000,"[164, 159, 162, 159, 162]",...,250,5,167,0.668000,156,0.624000,0.044000,"[160, 160, 161, 161, 161, 162, 162, 162, 162, ...","[0.64, 0.64, 0.644, 0.644, 0.644, 0.648, 0.648...",2024-12-21_12-25-11
9,9,gpt-35-turbo,"{'temperature': 0.0, 'seed': 12, 'top_p_k': 0.0}",navigate,"{'prompt_type': 'v2', 'shots': 'few'}",222,0.888000,250,1.000000,"[240, 240, 240, 240, 240]",...,250,5,240,0.960000,240,0.960000,0.000000,"[240, 240, 240, 240, 240, 240, 240, 240, 240, ...","[0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.96, 0.9...",2024-12-15_21-25-41


Parsing notes:



In [71]:
import pandas as pd
import os
from itertools import product
from collections import defaultdict


# Load the CSV file into a DataFrame
stability_df = pd.read_csv('stability_eval.csv')

# Sort the DataFrame by 'model' and 'task' columns
stability_df = stability_df.sort_values(by=['model', 'task', 'model_config'])

# Display the DataFrame

summary_d = defaultdict(list)
for model, task in product(set(stability_df['model']), 
                            set(stability_df['task'])):
    #print("-------------")
    #print(f"Model: {model}, Task: {task}:")
    mod_task_df = stability_df[(stability_df['model'] == model) 
                              & (stability_df['task'] == task)]
    mod_task_df = mod_task_df.sort_values(by=['model_config'])
    if len(mod_task_df.index) != 2:
        print(f"Expecting 2 entries in stability_output.csv: {model} x {task}")
        print(f"Got {len(mod_task_df.index)}, ignoring results")
        continue
    summary_d['model'].append(model)
    summary_d['task'].append(task)
    format_cols = set()
    for col in ['TARr', 'TARa', 'best_possible_accuracy', 
                'worst_possible_accuracy']:
        schema_val = mod_task_df[col].iloc[0]
        raw_val = mod_task_df[col].iloc[1]
        summary_d[f'{col}: schema - raw'].append(f"{schema_val:.1%} - {raw_val:.1%} = {schema_val- raw_val:.1%}")
        col_name = f'{col}'
        # summary_d[col_name].append(schema_val- raw_val)
        format_cols.add(col_name)
    
display(pd.DataFrame(summary_d).style.format({
    col: '{:.1%}' for col in format_cols
}))


# Display the DataFrame with the new column
#display(stability_df)


,model,task,TARr: schema - raw,TARa: schema - raw,best_possible_accuracy: schema - raw,worst_possible_accuracy: schema - raw
0,gpt-4o,college_mathematics,90.0% - 0.0% = 90.0%,90.0% - 61.0% = 29.0%,56.0% - 84.0% = -28.0%,49.0% - 52.0% = -3.0%
1,gpt-4o,high_school_european_history,98.8% - 15.2% = 83.6%,98.8% - 87.9% = 10.9%,89.1% - 86.7% = 2.4%,88.5% - 75.8% = 12.7%
2,gpt-4o,logical_deduction,97.2% - 41.2% = 56.0%,97.2% - 99.6% = -2.4%,96.0% - 100.0% = -4.0%,93.6% - 99.6% = -6.0%
3,gpt-4o,professional_accounting,94.3% - 9.6% = 84.8%,94.3% - 78.0% = 16.3%,79.1% - 83.3% = -4.3%,74.8% - 62.8% = 12.1%
4,gpt-4o,navigate,93.2% - 47.6% = 45.6%,93.2% - 99.6% = -6.4%,81.6% - 99.2% = -17.6%,74.8% - 98.8% = -24.0%
5,gpt-4o,ruin_names,98.8% - 1.2% = 97.6%,98.8% - 96.8% = 2.0%,91.6% - 91.2% = 0.4%,90.8% - 88.0% = 2.8%
6,gpt-4o,geometric_shapes,79.6% - 2.0% = 77.6%,79.6% - 68.8% = 10.8%,58.4% - 80.4% = -22.0%,47.6% - 56.4% = -8.8%
7,gpt-4o,public_relations,94.5% - 41.8% = 52.7%,94.5% - 91.8% = 2.7%,80.9% - 79.1% = 1.8%,76.4% - 73.6% = 2.7%
8,gpt-35-turbo,college_mathematics,89.0% - 85.0% = 4.0%,89.0% - 92.0% = -3.0%,33.0% - 36.0% = -3.0%,26.0% - 35.0% = -9.0%
9,gpt-35-turbo,high_school_european_history,97.6% - 98.2% = -0.6%,97.6% - 99.4% = -1.8%,83.0% - 81.2% = 1.8%,81.2% - 81.2% = 0.0%


Observations

Note that answer parsing is not perfect which means that the TARr scores are the only truly reliable measures. 

1. For `GPT-4o` 
    + The schema really helps TARr, big improvements across the tasks.
    + The schema mostly helps TARa with small drops for high TARa results like `logical_deduction` 97.2% - 99.6% = -2.4% and `navigate` 93.2% - 99.6% = -6.4%.
    + `best_possible_accuracy` sees sub 5% positive/negative impact for most tasks but considerable reduced performance for the schema for:
        - `college_mathematics` 56.0% - 84.0% = -28.0%
        - `navigate` 81.6% - 99.2% = -17.6%	
        - `geometric_shapes` 58.4% - 80.4% = -22.0%	
    + `worst_possible_accuracy` is more mixed
        - `navigate` was the only task with both ends of accuracy hurt by the use of schema-- best: 81.6% - 99.2% = -17.6%, worst: 74.8% - 98.8% = -24.0%
        - 
    
2. For `GPT-35-turbo` 